# Camada Bronze:
---

## Tabela Pagamentos:

### Objetivo:

Entender a padronização e comportamento dos dados para criar a view pagamentos_bronze a partir da camada RAW.

### Fonte:

`workspace.default.pagamentos`

### Destino:

`workspace.olist_bronze.pagamentos_bronze`

In [0]:
-- Garantindo a existência do SCHEMA para armazenamento:

CREATE SCHEMA IF NOT EXISTS workspace.olist_bronze;

In [0]:
-- Verificando a existência da tabela:
SELECT * FROM workspace.default.pagamentos;

In [0]:
-- Verificando quantas linhas tem na tabela:
SELECT COUNT(*) AS LINHAS 
FROM workspace.default.pagamentos;

In [0]:
-- Verificando os tipos de dados:
DESC workspace.default.pagamentos;

Os tipos de dados estão corretamente definidos, com exceção das colunas `payment_sequential` e `payment_installments`, que estão tipificadas como `BIGINT`. Considerando que o maior valor de ambas possui dois algarismos, esse tipo é desnecessariamente grande e resulta em uso ineficiente de memória. Dessa forma, a coluna pode ser convertida para `INT`, mantendo a semântica dos dados e reduzindo o consumo de memória.

In [0]:
-- Verificando missing values em cada coluna:
SELECT
      SUM(CASE WHEN order_id IS NULL THEN 1 ELSE 0 END) AS order_id_null,
      SUM(CASE WHEN payment_sequential IS NULL THEN 1 ELSE 0 END) AS payment_sequential_null,
      SUM(CASE WHEN payment_type IS NULL THEN 1 ELSE 0 END) AS payment_type_null,
      SUM(CASE WHEN payment_installments IS NULL THEN 1 ELSE 0 END) AS payment_installments_null,
      SUM(CASE WHEN payment_value IS NULL THEN 1 ELSE 0 END) AS payment_value_null
FROM workspace.default.pagamentos;

Não há valores nulos em nenhuma coluna.

In [0]:
-- Verificando valores inválidos em cada coluna:
SELECT 
      SUM(CASE WHEN order_id IS NOT NULL AND NOT order_id RLIKE '^[A-Za-z0-9]+$' THEN 1 ELSE 0 END) AS order_id_invalid,

      SUM(CASE WHEN payment_sequential IS NOT NULL AND NOT payment_sequential RLIKE '^[0-9]+$' THEN 1 ELSE 0 END) AS payment_sequential_invalid,

      SUM(CASE WHEN payment_type IS NOT NULL AND NOT payment_type RLIKE '^[A-Za-z_]+$' THEN 1 ELSE 0 END) AS payment_type_invalid,

      SUM(CASE WHEN payment_installments IS NOT NULL AND NOT payment_installments RLIKE '^[0-9]+$' THEN 1 ELSE 0 END) AS payment_installments_invalid,

      SUM(CASE WHEN payment_value IS NOT NULL AND NOT payment_value RLIKE '^[0-9]+(\.[0-9]+)?$' THEN 1 ELSE 0 END) AS payment_value_invalid

FROM workspace.default.pagamentos;

Não há valores inválidos em nenhuma coluna.

In [0]:
CREATE OR REPLACE VIEW workspace.olist_bronze.pagamentos_bronze AS
SELECT *

FROM workspace.default.pagamentos;